In [11]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

api_key = os.environ.get('riot_api_key')


def get_puuid(gameName = None, tagLine = None, api_key = None):

    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}'

    response = requests.get(link)

    return response.json()['puuid']


def get_name_and_tag(puuid=None, api_key=None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)

    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']

    return f'{gameName}#{tagLine}'

In [12]:

# response.json()

In [15]:
import pandas as pd

def get_leaderboard(top=None):
    root = 'https://br1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    grandmaster = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(f'{root}{chall}?api_key={api_key}')
    gm_response = requests.get(f'{root}{grandmaster}?api_key={api_key}')
    master_response = requests.get(f'{root}{master}?api_key={api_key}')

    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    grandmaster_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    master_df = pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    leaderboard = pd.concat([chall_df, grandmaster_df, master_df])[:top].reset_index(drop=True)
    leaderboard = leaderboard.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    leaderboard['rank'] += 1

    return leaderboard


In [16]:
get_leaderboard(100)

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,gPCqujeHBLdsZG7kUsTp7nPpaKsJd6Fnc--Fa2JKFcI0JQ...,525,43,22,False,False,True,True
1,2,8QP-WnGY3Na4cHUv6toZFep5Q3LjTsDSehAodZvZFQM9rA,514,42,24,False,False,True,True
2,3,euatw-KK17Bc_tBCPm04XWL0UTrRtWF8waHrhjHxqtQRXA,510,33,7,False,False,True,True
3,4,9tgI3rtg0WDun_0Lh79b_pxu7rNHboMAnX_qVCK_x2WuWDY,508,37,18,False,False,True,True
4,5,t8U8vpHS9KTYR8IsVGa5H6O8fkyapwUou36LigIbSOF9Qy4,550,50,22,False,False,True,True
...,...,...,...,...,...,...,...,...,...
95,96,sDRCSXM2KoIOEXAYnEdTyQUDx2rPVyvfMyRV_pNlLfE-ERY,139,38,32,False,False,True,False
96,97,HnOSCwiP2ZpMGTqtY5qqPdCW9qQfmono2wWTshKiYG7sww,136,31,16,False,False,True,True
97,98,PE7-QFsUZ0fRWIlu0YEjrcoADxAWiIYfbCKFKP2OkXoM__...,135,37,39,False,False,True,False
98,99,7MA0cZ3wZ65JRrKNKQ9XmNLJfvePSEP0okmKsB4OSL7rqDo,135,38,27,False,False,True,False
